In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import rebound
import reboundx
import time
from ctypes import cdll
from scipy.integrate import quad
from scipy.special import gamma, spence
from scipy.optimize import minimize_scalar
from numba import jit
import scipy.odr as odr
from Burkert_profile_functions import *
from NFW_profile_functions import *
from Sersic_profile_functions import *
import os
import shutil
import math

# matplotlib setup
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['figure.facecolor'] = 'white'
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif',size=15)
plt.rc('axes', linewidth=1.5)
plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize=15, direction='in')
plt.rc('ytick', labelsize=15, direction='in')
plt.rc('xtick', top=True)
plt.rc('ytick', right=True)
plt.rc('xtick.minor', visible=True)
plt.rc('ytick.minor', visible=True)
plt.rc('xtick.major',size=10, pad=4)
plt.rc('xtick.minor',size=5, pad=4)
plt.rc('ytick.major',size=10)
plt.rc('ytick.minor',size=5)
plt.rc('legend', fontsize=15)

## masses of the GCs from the Georgiev catalogue in units of 1e5 solar masses
UGC7369_masses = np.array([ 0.5270876 ,  0.15333488,  1.08466861,  0.23894274,  0.53477108,
         0.66067185,  2.74824974, 43.7230426 ,  0.25702028,  0.68325756,
         0.48945594,  0.78176033,  0.98889862,  0.70596634,  1.74467125,
         1.42423357,  1.17664156,  0.49028781,  0.71836984,  0.42620355,
         0.33863009])
## total mass in GCs

## a C implementation of friction and background forces in rebound, for speed
clib_additional_forces = cdll.LoadLibrary("/home/vk9342/spring2024research/additional_forces.so")

In [2]:
#So rho_0 and r_0 are going to depend on the mass and concentration of the halo, denoted by c
#get concentrations from mass is Diemer & Joyce 2019
#https://iopscience.iop.org/article/10.3847/1538-4357/aafad6
#The relevant figure is the top middle panel of figure 3. 
#Since we run the simulation for 10 Gyr, this corresponds to redshift approximately 2, so we should be looking at the orange curve


#get concentrations as a function of halo mass. as an input, 
#you should use the halo mass in units of solar masses

# a fitting function to Diemer & Joyce 2019 z = 2 c(M):
#it’s a “piecewise-power-law fit to the concentration-mass relation at z = 2 from Diemer & Joyce (2019)”
def c_func(M_halo):
    c_values = np.array([5.606078101846313, 5.505601101255639, 5.296315905448495, 5.094986297483347, 4.901309860470804, 4.666519328282937, 4.500743526125894, 4.329656123893253, 4.154325163981534, 3.975808982640419, 3.834570325527022, 3.727126007769686, 3.6040211391832533, 3.512099050898484, 3.4225214744765236, 3.4402523213171285])
    scaled_M_values = np.array([1208795796.6963477, 1628413544.0132532, 2799360456.643171, 5080218046.9130125, 8973072494.285637, 18147780986.393196, 31197345819.126064, 52197182204.356415, 97327438615.81503, 191581222925.96384, 347677407476.5763, 566162599282.2716, 1114445470753.5625, 2193696137571.7903, 5976825664072.4, 14221361511653.348])
    scaled_M = M_halo/0.7
    return 10**np.interp(np.log10(scaled_M), np.log10(scaled_M_values), np.log10(c_values))


# NFW halo - eqs (3) and (4)

def r_0_func (M_halo):  ## in kpc
    rho_crit = 136 #Solar masses / kpc^3
    c = c_func(M_halo)
    # c = 5
    r_0 = 1/c * (3*M_halo/ (800 * math.pi * rho_crit))**(1/3)
    return (r_0)


def rho_0_func (M_halo):  # in 10^5 solar masses / kpc^3 ?????
    rho_crit = 0.00136 # 10^5 Solar masses / kpc^3
    c = c_func(M_halo)
    # c = 5
    rho_0 = 200/3 * c**3 * rho_crit / (np.log(1+c) - c/(1+c))
    return (rho_0)


In [3]:
# mass function: gaussian in logM, mean/variance taken from Georgiev catalogue normal dist fit
def mass_function(logmass):
    # currently using a NSC cut fit
    mean = 4.987311450993631 - 5
    variance = 0.18871641457327373
    return 1/np.sqrt(2*np.pi*variance) * np.exp(-(logmass-mean)**2 / (2*variance))

# custom merge function
def merge_energy(sim_pointer, collided_particles_index):
    sim = sim_pointer.contents
    ps = sim.particles
    
    i = collided_particles_index.p1
    j = collided_particles_index.p2
    
    # calculating the relevant energy to see if a merger really should happen or not
    dp = ps[i]-ps[j]
    total_mass = ps[i].m + ps[j].m
    reduced_mass = (ps[i].m*ps[j].m)/total_mass
    r_12 = np.sqrt(dp.x**2 + dp.y**2 + dp.z**2)
    v_12sq = dp.vx**2 + dp.vy**2 + dp.vz**2
    U_12 = -sim.G*ps[i].m*ps[j].m / (r_12**2.11 + (1.7*sim.softening)**2.11)**(1/2.11)
    E_12 = 0.5*reduced_mass*v_12sq + U_12
    #print('merge function called')
    if E_12 < 0:
        # if clusters did meet the energy condition, delete particle j and update particle i to the merged one
        #print(np.sqrt(ps[i].x**2 + ps[i].y**2 + ps[i].z**2), dp.vx, dp.vy, dp.vz)
        old_radius = ps[i].r
        new_particle = (ps[i]*ps[i].m + ps[j]*ps[j].m)/total_mass
        ps[i] = new_particle
        ps[i].m = total_mass
        ps[i].r = 2*2.548e-3*(total_mass/0.1)**(0.242)
        return 2
    else:
        # in case the clusters did pass near each other, but didn't meet the energy condition, do nothing
        return 0

In [4]:
#NFW
R_e_values = [3.1581672,  2.86368325, 2.9590771,  3.08617562, 3.54959467, 3.27100949,
 2.9558744,  3.12399573, 3.00030197, 2.99788207, 3.15393355, 2.94114023,
 3.39039245, 3.23279048, 3.13641168, 3.14609655, 3.08135367, 3.33980461,
 3.23735857, 3.43498177]
n_values = [3]
GC_masses = [6.15799138, 5.37559416, 6.24233033, 5.76468838, 6.76248489, 5.66982948,
 5.43773922, 6.01390964, 6.59337933, 7.17866832, 6.56967568, 6.74838429,
 5.52873797, 6.79590216, 6.4864891,  6.02153588, 6.46528175, 5.47929711,
 5.73249753, 6.22083521]
halo_masses = [466996354113.9495, 315315013599.5077, 355576925103.83215, 28451463450.660877, 1338940289706.7505, 219087984455.99396, 51609988226.88336, 54562920746.327385, 219340495386.03568, 172093398057.08545, 563188191144.1494, 75744898855.61324, 42862243580.17732, 2219758000720.6353, 543906676689.0823, 96088242499.56992, 49412724441.5778, 103696101689.54207, 66430278015.64493, 62191618612.12596]


# rho_0_NFW = 17.99 # 10^5 solar masses / kpc^3
# r_0_NFW = 25.34 # kpc

n = 3
for i in range (1):
    R_e = R_e_values[0]
    GC_mass = GC_masses[0]
    halo_mass = halo_masses[0]
    rho_0_NFW = rho_0_func(halo_mass)
    r_0_NFW = r_0_func(halo_mass)
    # NFW ergodic distribution function, calling helpers from NFW_profile_functions.py
    ######################################
    # MAKE SURE THESE rho_0 and r_0 ARE THE SAME AS THE ONES IN THE ADDITIONAL_FORCES FILE
    ######################################

    G = 0.449 # (kpc/Gyr)^2 * kpc / 10^5 solar masses
    Sersic_params_NFW = [R_e, n]
    @np.vectorize
    def NFW_ergodic_f(E):
        def integrand(psi):
            def r_psi(psi):
                rootfinding_func = lambda r: np.abs(psi + phi_NFW(r, rho_0_NFW, r_0_NFW))  #lambda (def short function) arguments : expression
                return minimize_scalar(rootfinding_func, method='bounded', bounds=[1e-20, 500]).x
            return np.sqrt(-E-psi)*d3rhodpsi3_NFW(r_psi(psi), rho_0_NFW, r_0_NFW, Sersic_params_NFW[0], Sersic_params_NFW[1])
        return 1/(np.sqrt(2)*np.pi**2) * quad(integrand, 0, -E)[0]

    start = time.time()
    print('Calculating NFW Energies and Weights')
    energy_values_NFW = 4 * np.pi * G * rho_0_NFW * r_0_NFW**2 * np.arange(-1+0.005, -0.005, 0.005)
    energy_weights_NFW = NFW_ergodic_f(energy_values_NFW)
    #E_dist_NFW = [energy_values_NFW, energy_weights_NFW]
    print('Finished calculating NFW Energies and Weights')
    print(time.time()-start)
    print(n, R_e)

    #making interpolated functions for f(E) that can be called in the initialization process for speed

    def NFW_f_interp(E):
        return 10**np.interp(E, energy_values_NFW, np.log10(energy_weights_NFW))


    #################################
    ## SAVES OUTPUTS FOR EVERY 0.5 Gyr

    def run_sim(potential, force, halo_params, ergodic_f, seed, t, r_0, rho_0):
        start = time.time()
        np.random.seed(seed)

        # usual initialization of the sim, units, etc.
        sim = rebound.Simulation()
##############
        sim.particles[0].params = [r_0, rho_0]
##############        
        sim.ri_ias15.epsilon = 0 # using fixed timesteps
        #sim.ri_ias15.min_dt = 1e-5
        #sim.ri_ias15.max_dt = 1e-3
        sim.dt = 5e-4 # fine to use this for quick tests, for the actual paper use the min & max above
        sim.G = 0.449 # setting units to kpc, Gyr, 10^5 M_sun
        sim.softening = 0.01 # plummer softening of 10 pc
        sim.collision = "line" # counting collisions as linear interpolations between timesteps
        sim.collision_resolve = merge_energy # conserving mass and momentum, with energy condition for mergers

        # Sersic radial distribution and values to draw positions with their weights

        pos_values = np.arange(1e-4, 10, 1e-4)
        pos_weights = nu(pos_values, R_e, n)*4*np.pi*pos_values**2
        pos_weights /= sum(pos_weights)


        # setting up a mass function gaussian in logM
        logmass_values = np.linspace(-2, 2, 1000)
        logmass_weights = mass_function(logmass_values)
        logmass_weights /= sum(logmass_weights)

        total_GC_mass = 0
        # add particles w/ randomly drawn m, r, v until we reach the UGC7369 total GC mass
        while total_GC_mass < GC_mass-10**(4.987311450993631 - 5):
            mass = np.exp(10/23) * 10**np.random.choice(logmass_values, p=logmass_weights) #random value is sampled from the logmass_values array with probabilities defined by the logmass_weights
            pos_rad = np.random.choice(pos_values, p=pos_weights)
            pos_theta = np.random.uniform(0, np.pi)
            pos_phi = np.random.uniform(0, 2*np.pi)
            x_i = pos_rad*np.sin(pos_theta)*np.cos(pos_phi)  #Cartesian coordinates
            y_i = pos_rad*np.sin(pos_theta)*np.sin(pos_phi)
            z_i = pos_rad*np.cos(pos_theta)

            # drawing velocities from provided distribution
            vel_values = np.arange(0.01, 100, 0.01)
            vel_weights = 4*np.pi*vel_values**2 * \
                            ergodic_f(0.5*vel_values**2 + potential(pos_rad, rho_0_NFW, r_0_NFW))/(nu(pos_rad, R_e, n))
            vel_weights /= sum(vel_weights)
            vel_mag = np.random.choice(vel_values, p=vel_weights)
            v_theta = np.random.uniform(0, np.pi)
            v_phi = np.random.uniform(0, 2*np.pi)
            vx_i = vel_mag * np.sin(v_theta)*np.cos(v_phi)
            vy_i = vel_mag * np.sin(v_theta)*np.sin(v_phi)
            vz_i = vel_mag * np.cos(v_theta)

            sim.add(m = mass, r = 2*2.548e-3*(mass/0.1)**(0.242), \
                    x = x_i, y = y_i, z = z_i, vx = vx_i, vy = vy_i, vz = vz_i)
            total_GC_mass += mass/np.exp(10/23)

        rebx = reboundx.Extras(sim)
        # adding in the additional forces to work with reboundx
        myforce = rebx.create_force("additional_forces")
        myforce.force_type = "vel"
        myforce.update_accelerations = force
        rebx.add_force(myforce)
        # setting up built-in exponential mass loss
        modifymass = rebx.load_operator("modify_mass")
        rebx.add_operator(modifymass)
        # setting the mass-loss timescales of each particle, tau chosen to match linear decrease of 1/3 by end
        for p in sim.particles:
            p.params["tau_mass"] = -23

        # keeping track of initial positions, velocities, and masses
        initial_masses = []
        initial_positions = []
        initial_vels = []

        for p in sim.particles:
            initial_masses.append(p.m)
            initial_positions.append(np.array([p.x, p.y, p.z]))
            initial_vels.append(np.array([p.vx, p.vy, p.vz]))

        # integrating
        sim.integrate(t)

        # keeping track of final positions, velocities, and masses
        final_masses = []
        final_positions = []
        final_vels = []

        for p in sim.particles:
            final_masses.append(p.m)
            final_positions.append(np.array([p.x, p.y, p.z]))
            final_vels.append(np.array([p.vx, p.vy, p.vz]))

        initial_masses, final_masses = np.array(initial_masses), np.array(final_masses)
        initial_positions, final_positions = np.array(initial_positions), np.array(final_positions)
        initial_vels, final_vels = np.array(initial_vels), np.array(final_vels)

        # clearing sim from memory now that we've stored everything important
        sim = None

        # returning 3 sets of 2 things: the initial and final masses, positions, and velocities
        return [initial_masses, final_masses], [initial_positions, final_positions], [initial_vels, final_vels]

    ###############################################################################
    ## run N_iterations versions of the sim, outputting every 0.5 Gyrs for 10 Gyrs

    def run_iterations(N_iterations, output_name, potential, force, halo_params, ergodic_f):
        seeds = np.random.randint(1, 4294967295, size = N_iterations)

        start = time.time()

        for output_time, t in enumerate(np.arange(0, 10.5, 0.5)):

            if output_time == 0:
                # save initial data
                f = open('Sim_'+str(n)+'_'+output_name+'_'+str(output_time)+'_'+str(R_e)+'.txt', 'w')
                f.write('t = '+str(t)+' Gyr \n')
                f.write('iteration m x y z vx vy vz n Re \n')
                for k in range(N_iterations):
                    seed = seeds[k]
                    masses, positions, velocities = run_sim(potential = potential, force = force, \
                                                            halo_params = halo_params, ergodic_f = ergodic_f, \
                                                            seed = seed, t=0.001)
                    for GC_n in range(len(masses[0])):
                        f.write(str(k)+' '+str(masses[0][GC_n])+' '+str(positions[0][GC_n][0])+' '\
                                +str(positions[0][GC_n][1])+' '+str(positions[0][GC_n][2])+' '\
                                +str(velocities[0][GC_n][0])+' '+str(velocities[0][GC_n][1])+' '\
                                +str(velocities[0][GC_n][2])+' '+str(n)+' '+str(R_e)+'\n')
                f.close()
            else:
                # save final data
                f = open('Sim_' +output_name+'_'+str(output_time)+'_'+str(R_e)+str(GC_mass)+str(halo_mass)+'.txt', 'w')
                f.write('t = '+str(t)+' Gyr \n')
                f.write('iteration m x y z vx vy vz n Re \n')
                for k in range(N_iterations):
                    seed = seeds[k]
                    masses, positions, velocities = run_sim(potential = potential, force = force, \
                                                            halo_params = halo_params, ergodic_f = ergodic_f, \
                                                            seed = seed, t=t)
                    for GC_n in range(len(masses[1])):
                        f.write(str(k)+' '+str(masses[1][GC_n])+' '+str(positions[1][GC_n][0])+' '\
                                    +str(positions[1][GC_n][1])+' '+str(positions[1][GC_n][2])+' '\
                                    +str(velocities[1][GC_n][0])+' '+str(velocities[1][GC_n][1])+' '\
                                    +str(velocities[1][GC_n][2])+' '+str(n)+' '+str(R_e)+'\n')
                f.close()
            print('finished until t = '+str(t), str(time.time()-start))

    ##############################################
    #RUNS EVERYTHING 

    run_iterations(10, 'NFW', phi_NFW, clib_additional_forces.NFW_DF_Petts, [17.99, 25.34], NFW_f_interp, r_0, rho_0)
    data_initial = np.genfromtxt('Sim_'+'NFW_0'+'_'+str(R_e)+str(GC_mass)+str(halo_mass)+'.txt', skip_header=2)
    initial_masses = data_initial[:, 1]
    initial_radii = np.sqrt(data_initial[:, 2]**2 + data_initial[:, 3]**2 + data_initial[:, 4]**2)
    data_final = np.genfromtxt('Sim_'+'_NFW_20'+'_'+str(R_e)+str(GC_mass)+str(halo_mass)+'.txt', skip_header=2)
    final_masses = data_final[:, 1]
    final_radii = np.sqrt(data_final[:, 2]**2 + data_final[:, 3]**2 + data_final[:, 4]**2)

    plt.plot([1e-2, 1e2], [0.9, 0.9], 'k--')
    plt.scatter(initial_masses, initial_radii, color='gray', s=5)
    plt.scatter(final_masses, final_radii, color='lightcoral', s=5)
    #plt.scatter(UGC7369_masses, UGC7369_rperps, color='b', s=5)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim(min(final_masses)*0.9, max(final_masses)*1.5)
    plt.title('Re='+str(R_e)+' n='+str(n))
    # plt.savefig('NFW-3_'+str(n)+'_'+str(R_e)+'.png')
    plt.show()



Calculating NFW Energies and Weights
Finished calculating NFW Energies and Weights
4.6110944747924805
3 3.1581672
finished until t = 0.0 0.4691343307495117
finished until t = 0.5 1.2968578338623047
finished until t = 1.0 2.8120803833007812
finished until t = 1.5 4.984634876251221


KeyboardInterrupt: 

In [14]:
def halo_mass_calc (M_star):
    alpha = 1.74
    M1 = 10**(12.5) #solar masses
    log_epsilon = -1.70
    M = M1 * 10**(1/alpha * (np.log10(M_star/M1) - log_epsilon))
    return M

M200 = halo_mass_calc(1.72 * 10**8)
print(M200/10**10)
h = 0.7
c = 5

M_star1 = 1.63 * 10**7
M_star2 = 1.72 * 10**8
M_star3 = 1.73 * 10**9


rho_crit = 0.00136 # critical density with h = 0.7 in units of 10^5/kpc^3
# NFW scale radius from M200 and c
r0_N = 1/c * (3*(M200/10**5)/(4*np.pi*200*rho_crit))**(1/3)
# NFW scale density from c
rho0_N = rho_crit * 200/3 * (c**3 / (np.log(1+c) - c/(1+c)))

print (r0_N, rho0_N)

10.621008932359809
19.53736915883254 11.824941859242292


In [ ]:
rho value (11.82) r0 19.54